
# Montgomery County, MD - Data Exploration and Cleaning
## Target = 'Street name'
### Dataset Source: [dataMontgomery](https://data.montgomerycountymd.gov/Public-Safety/Crime/icn6-v9z3)

Initially, I chose City as my target, which would of made it a multi-variate classsificiation. I decided to make it a binary classification, choosing to only predict crime in Silver Spring.

As I delved deeper into my analysis, I found my target unsatifactory. So I change the target to [Crime_Type].

I decided to predict the type of crime in montgomery county, MD

I was motivated to do this dataset and specific target because I wanted to show my little sister the cool stuff I could do with this skill. We lived most of our childhood in this city, so she wanted to know how 'dangerous' it was. Then, she wanted me to predict if someone would be murdered the next day, and so, I had to explain to her that I cant quite do that.

### Column Description of Original Dataset
ID - Police Incident Number

Offense Code - Offense_Code is the code for an offense committed within the incident as defined by the National Incident-Based Reporting System (NIBRS) of the Criminal Justice Information Services (CJIS) Division Uniform Crime Reporting (UCR) Program.

CR Number - Police Report Number

Dispatch Date/Time - The actual date and time a Officer was dispatched

NIBRS Code - FBI NIBRS codes: National Incident-Based Reporting System (NIBRS) is an incident-based reporting system used by law enforcement agencies in the United States for collecting and reporting data on crimes. Local, state and federal agencies generate NIBRS data from their records management systems.

Victims - Number of Victims

Crime Name1 - Crime against Society/Person/Property or Other

Crime Name2 - Describes the NIBRS_CODE

Crime Name3 - Describes the OFFENSE_CODE

Police District Name - Name of District (Rockville,Weaton etc.)

Block Address - Address in 100 block level

City - City Name

State - State(Maryland)

Zip Code - Zip code

Agency - Assigned Police Department

Place - Place description

Sector - Police sector name, a subset of District

Beat - Police patrol area, a subset of Sector

PRA - Police Response Area, a subset of Beat

Address Number - House or Business Number

Street Prefix - North, South, East, West

Street Name - Name of the Street

Street Suffix - Quadrant (NW, SW, etc)

Street Type - Ave, Drive, Road, etc

Start_Date_Time - Occurred from date/time

End_Date_Time - Occurred to date/time

Latitude - Latitude Coordinate

Longitude - Longitude Coordinate

Police District Number - Major Police Boundary

Location - Location Coordinate

In [1]:

# import pandas as pd 
import os

# All imports 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import pandas_profiling as pp

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer 
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import tree
from sklearn.model_selection import cross_val_score

# scipy
import scipy.stats as st
from scipy.stats import uniform, randint

from sklearn import tree
from sklearn.model_selection import cross_val_score

# pipeline
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from category_encoders.ordinal import OrdinalEncoder

# plotly
import plotly.express as px

# date-time
from datetime import datetime

# eli5
import eli5
from eli5.sklearn import PermutationImportance

# pdp
from pdpbox import pdp

In [2]:
# XGB
from xgboost.sklearn import XGBClassifier  
from xgboost.sklearn import XGBRegressor
import xgboost as xgb

In [3]:
pwd

'/Users/yonipineda/lambda_folder/montgomery-crime-project/Montgomery-Project/notebooks'

In [4]:
# Set pandas's max row display
pd.set_option('display.max_row',1000)

# Set pandas's max column width to 50
pd.set_option('display.max_columns',200)

# Set Seaborn style to default
sns.set_style('darkgrid')

In [230]:
# load df
df = pd.read_csv('Crime.csv')

In [10]:
# profile 
import pandas_profiling

profile_report = df.profile_report(minimal = True)

profile_report

# Feature Engineering,  Cleaning ,  and wrangling data

In [231]:
# check data
df.head()

,Incident ID,Offence Code,CR Number,Dispatch Date / Time,NIBRS Code,Victims,Crime Name1,Crime Name2,Crime Name3,Police District Name,Block Address,City,State,Zip Code,Agency,Place,Sector,Beat,PRA,Address Number,Street Prefix,Street Name,Street Suffix,Street Type,Start_Date_Time,End_Date_Time,Latitude,Longitude,Police District Number,Location
0,201087097,5707,16033232,NaN,90J,1,Crime Against Society,Trespass of Real Property,TRESPASSING,WHEATON,13700 BLK CONNECTICUT AVE,ROCKVILLE,MD,20906.0,MCPD,Retail - Drug Store/Pharmacy,K,4K2,337,13700.0,NaN,CONNECTICUT,NaN,AVE,07/01/2016 12:04:00 AM,NaN,39.078911,-77.080827,4D,"(39.0789, -77.0808)"
1,201215730,5311,180058531,11/22/2018 04:58:01 AM,90C,1,Crime Against Society,Disorderly Conduct,PUBLIC PEACE - DISORDERLY CONDUCT,CITY OF TAKOMA PARK,6800 BLK NEW HAMPSHIRE AVE,NaN,MD,20912.0,TPPD,Parking Lot - Commercial,T,8T3,808,6800.0,NaN,NEW HAMPSHIRE,NaN,AVE,11/22/2018 04:58:00 AM,11/22/2018 10:00:00 AM,38.973022,-76.996799,8D,"(38.973, -76.9968)"
2,201229073,3562,190009928,03/03/2019 04:59:49 AM,35A,1,Crime Against Society,Drug/Narcotic Violations,DRUGS - MARIJUANA - POSSESS,BETHESDA,5600 BLK OVERLEA RD,BETHESDA,MD,20816.0,MCPD,Street - Residential,E,2E1,3,5600.0,NaN,OVERLEA,NaN,RD,03/03/2019 04:52:00 AM,03/03/2019 04:57:00 AM,38.956840,-77.111362,2D,"(38.9568, -77.1114)"
3,201233523,1114,190015440,04/03/2019 11:53:15 AM,11B,1,Crime Against Person,Forcible Sodomy,SEX ASSAULT - SODOMY-GIRL-STRONG-ARM,SILVER SPRING,1 BLK UNIVERSITY BLV E,SILVER SPRING,MD,20901.0,MCPD,School/College,H,3H2,156,1.0,NaN,UNIVERSITY,E,BLV,04/02/2019 11:00:00 AM,04/02/2019 12:00:00 PM,39.020392,-77.012776,3D,"(39.0204, -77.0128)"
4,201087102,3562,16033238,NaN,35A,1,Crime Against Society,Drug/Narcotic Violations,DRUGS - MARIJUANA - POSSESS,SILVER SPRING,900 BLK SILVER SPRING AVE,SILVER SPRING,MD,20910.0,MCPD,Parking Lot - County,G,3G1,93,900.0,NaN,SILVER SPRING,NaN,AVE,07/01/2016 12:58:00 AM,07/01/2016 12:58:00 AM,38.991701,-77.024096,3D,"(38.9917, -77.0241)"


In [232]:
# wrangling function

def wrangle(X):
    
    # We will rename each of the three Crime Columns to better describe their function
    X = X.rename(columns={'Crime Name1' : 'Crime_Type',
                        'Crime Name2' : 'NIBR_Crime', 
                        'Crime Name3' : 'OffCode_Crime'})
    
    
    # to datetime
    X['Start_Date_Time'] = pd.to_datetime(X['Start_Date_Time'])
    col = ['End_Date_Time']
    X = X.drop(columns=col)
    
    # creating individual columns for sec, min, hour, day, month and, year for plotting
    X["Second"] = X["Start_Date_Time"].map(lambda x: x.second)
    X["Minute"] = X["Start_Date_Time"].map(lambda x: x.minute)
    X["Hour"] = X["Start_Date_Time"].map(lambda x: x.hour)
    X["Week"] = X["Start_Date_Time"].map(lambda x: x.week)
    X["Day"] = X["Start_Date_Time"].map(lambda x: x.day)
    X["Month"] = X["Start_Date_Time"].map(lambda x: x.month)
    X["Year"] = X["Start_Date_Time"].map(lambda x: x.year)
    
   
    
    # drop incident id
    id_col = ['Incident ID']
    X = X.drop(columns=id_col)
    
    return X

df = wrangle(df)

In [233]:
limit = 10
def wrangle(X):
    # combine everything under the limit into NearZero - Combo
    # NearZero - Combo is every place under top10
    top_10 = X['Place'].value_counts()[:limit].index
    
    X.loc[~X['Place'].isin(top_10), 'Place'] = 'Other Place'
    
    # Do the same for OffCode_Crime
    OCC = X['OffCode_Crime'].value_counts()[:limit].index
    X.loc[~X['OffCode_Crime'].isin(OCC), 'OffCode_Crime'] = 'Other Types Of Crime'
    
    # Do the same for NIBR_Crime
    nibr = X['NIBR_Crime'].value_counts()[:limit].index
    X.loc[~X['NIBR_Crime'].isin(nibr), 'NIBR_Crime'] = 'Other Types Of Crime'
    return X
df = wrangle(df)

In [234]:
def wrangle(X):
    
    limit = 10
    
    # Combine under top10 in street as 'Other Streets'
    street = X['Street Name'].value_counts()[:limit].index
    X.loc[~X['Street Name'].isin(street), 'Street Name'] = 'Other Street'
 
    
    # Combine under top10 in Block Address as 'Other Block Adress'
    block = X['Block Address'].value_counts()[:limit].index
    X.loc[~X['Block Address'].isin(block), 'Block Address'] = 'Other Block Address'
    
    return X

df = wrangle(df)

In [235]:
def wrangle(X):
    limit = 5
    
    # combine under top10 in City as 'Other Cities'
    c = X['City'].value_counts()[:limit].index
    X.loc[~X['City'].isin(c), 'City'] = 'Other Cities'
    
    return X
df = wrangle(df)

In [236]:
# fill missing value
df = df.fillna(method='pad')

In [237]:
# drop remaining nans
df = df.dropna()

In [238]:
# checking df
df.head()

,Offence Code,CR Number,Dispatch Date / Time,NIBRS Code,Victims,Crime_Type,NIBR_Crime,OffCode_Crime,Police District Name,Block Address,City,State,Zip Code,Agency,Place,Sector,Beat,PRA,Address Number,Street Prefix,Street Name,Street Suffix,Street Type,Start_Date_Time,Latitude,Longitude,Police District Number,Location,Second,Minute,Hour,Week,Day,Month,Year
90,1399,190051543,10/26/2019 03:38:45 PM,13B,1,Crime Against Person,Simple Assault,ASSAULT - 2ND DEGREE,ROCKVILLE,Other Block Address,ROCKVILLE,MD,20850.0,RCPD,School/College,A,1A3,255,1.0,W,Other Street,E,AVE,2019-10-14 09:00:00,39.084184,-77.152957,1D,"(39.0842, -77.153)",0,0,9,42,14,10,2019
91,2399,190051387,10/25/2019 08:17:07 PM,23H,1,Crime Against Property,All other Larceny,LARCENY (DESCRIBE OFFENSE),MONTGOMERY VILLAGE,Other Block Address,Other Cities,MD,20886.0,MCPD,Other Place,R,6R1,505,18700.0,W,Other Street,E,AVE,2019-10-25 18:00:00,39.163474,-77.203254,6D,"(39.1635, -77.2033)",0,0,18,43,25,10,2019
92,5403,190055512,11/17/2019 07:04:58 PM,90D,1,Crime Against Society,Driving Under the Influence,Other Types Of Crime,SILVER SPRING,Other Block Address,SILVER SPRING,MD,20910.0,MCPD,Street - Residential,G,3G2,175,18700.0,W,Other Street,E,RD,2019-11-17 19:04:00,39.015332,-77.049200,3D,"(39.0153, -77.0492)",0,4,19,46,17,11,2019
93,5404,190055512,11/17/2019 07:04:58 PM,90D,1,Crime Against Society,Driving Under the Influence,DRIVING UNDER THE INFLUENCE LIQUOR,SILVER SPRING,Other Block Address,SILVER SPRING,MD,20910.0,MCPD,Street - Residential,G,3G2,175,18700.0,W,Other Street,E,RD,2019-11-17 19:04:00,39.015332,-77.049200,3D,"(39.0153, -77.0492)",0,4,19,46,17,11,2019
94,9113,200000358,01/03/2020 07:11:01 AM,90Z,1,Other,All Other Offenses,MENTAL ILLNESS - EMERGENCY PETITION,SILVER SPRING,Other Block Address,SILVER SPRING,MD,20904.0,MCPD,Residence - Apartment/Condo,I,3I1,145,11500.0,W,Other Street,E,LA,2020-01-03 07:11:00,39.045501,-76.983217,3D,"(39.0455, -76.9832)",0,11,7,1,3,1,2020


In [239]:
# Checking Shape
df.shape

(207139, 35)

In [240]:
# check value counts for city
df['City'].value_counts()

SILVER SPRING    73328
Other Cities     41172
GAITHERSBURG     29615
ROCKVILLE        28956
GERMANTOWN       20277
BETHESDA         13791
Name: City, dtype: int64

In [241]:
# check value counts for months 
df['Year'].value_counts(ascending=True)

2020    17576
2016    28180
2019    51010
2018    54171
2017    56202
Name: Year, dtype: int64

In [242]:
# Looking at observations only after 2020
twnty = df[df['Year'] >= 2020]
twnty.head()

# check how many months this dataset covers in 2020
months_2020 = twnty['Month'].value_counts(ascending=True)

# print 
print(f"Months covered in 2020: \n{months_2020}")


'''
When I originally did this project, sometime in Feb,2020: it only covered a up until jan 5th. 
Pretty cool that they update
'''

Months covered in 2020: 
4    2753
5    2838
3    3819
2    4082
1    4084
Name: Month, dtype: int64


'\nWhen I originally did this project, sometime in Feb,2020: it only covered a up until jan 5th. \nPretty cool that they update\n'

In [243]:
def leakage(X):
    
    # Drop columns the leak information into the predictive model
    
    leak = ['Offence Code', 'Location', 'CR Number', 'NIBRS Code','NIBR_Crime', 'OffCode_Crime']
    
    X = X.drop(columns=leak)
    
    return X
df= leakage(df)

In [244]:
# drop start date/time
bye = ['Start_Date_Time', 'Dispatch Date / Time']
df = df0.drop(columns=bye)

In [245]:
# Check dataframe once more 
shape = df.shape
print(f"Data Shape: {shape}")
df.head()

Data Shape: (207139, 27)


,Victims,Crime_Type,Police District Name,Block Address,City,State,Zip Code,Agency,Place,Sector,Beat,PRA,Address Number,Street Prefix,Street Name,Street Suffix,Street Type,Latitude,Longitude,Police District Number,Second,Minute,Hour,Week,Day,Month,Year
90,1,Crime Against Person,ROCKVILLE,Other Block Address,ROCKVILLE,MD,20850.0,RCPD,School/College,A,1A3,255,1.0,W,Other Street,E,AVE,39.084184,-77.152957,1D,0,0,9,42,14,10,2019
91,1,Crime Against Property,MONTGOMERY VILLAGE,Other Block Address,Other Cities,MD,20886.0,MCPD,Other Place,R,6R1,505,18700.0,W,Other Street,E,AVE,39.163474,-77.203254,6D,0,0,18,43,25,10,2019
92,1,Crime Against Society,SILVER SPRING,Other Block Address,SILVER SPRING,MD,20910.0,MCPD,Street - Residential,G,3G2,175,18700.0,W,Other Street,E,RD,39.015332,-77.049200,3D,0,4,19,46,17,11,2019
93,1,Crime Against Society,SILVER SPRING,Other Block Address,SILVER SPRING,MD,20910.0,MCPD,Street - Residential,G,3G2,175,18700.0,W,Other Street,E,RD,39.015332,-77.049200,3D,0,4,19,46,17,11,2019
94,1,Other,SILVER SPRING,Other Block Address,SILVER SPRING,MD,20904.0,MCPD,Residence - Apartment/Condo,I,3I1,145,11500.0,W,Other Street,E,LA,39.045501,-76.983217,3D,0,11,7,1,3,1,2020


In [246]:
# more exploration
addy = df['Street Name'].value_counts(ascending=False)
pprint(addy)

Other Street     167328
GEORGIA            9665
FREDERICK          6046
VEIRS MILL         5005
NEW HAMPSHIRE      3935
UNIVERSITY         3888
ROCKVILLE          3644
COLESVILLE         2427
CONNECTICUT        1765
RANDOLPH           1738
RUSSELL            1698
Name: Street Name, dtype: int64


In [247]:
limit = 167328
other = df['Street Name'].value_counts()[:1].index
not_other = df['Street Name'].value_counts()[1:].index
print(f"Streets with only 10 observations in them: {other}\n  \nStreets I want: {not_other}")

Streets with only 10 observations in them: Index(['Other Street'], dtype='object')
  
Streets I want: Index(['GEORGIA', 'FREDERICK', 'VEIRS MILL', 'NEW HAMPSHIRE', 'UNIVERSITY',
       'ROCKVILLE', 'COLESVILLE', 'CONNECTICUT', 'RANDOLPH', 'RUSSELL'],
      dtype='object')


In [248]:
# drop 'Other Street'. Its a compiliation of addresses with less than 10 unique observations 
streets = ['GEORGIA', 'FREDERICK', 'VEIRS MILL',
           'NEW HAMPSHIRE', 'UNIVERSITY', 'ROCKVILLE', 'COLESVILLE', 'RANDOLPH', 'RUSSELL']

df = df[df['Street Name'].isin(streets)]

In [249]:
# check to make sure I got what I want
s = df.shape
print(f"Checking Shape with new changes: {s}")
df.head()

'''
Hmm, thats definitely more observations without applying the function below this: remove_other.
This is enough observations for this project.
'''

Checking Shape with new changes: (38046, 27)


,Victims,Crime_Type,Police District Name,Block Address,City,State,Zip Code,Agency,Place,Sector,Beat,PRA,Address Number,Street Prefix,Street Name,Street Suffix,Street Type,Latitude,Longitude,Police District Number,Second,Minute,Hour,Week,Day,Month,Year
99,1,Other,MONTGOMERY VILLAGE,700 BLK RUSSELL AVE,GAITHERSBURG,MD,20877.0,MCPD,Parking Lot - Commercial,R,6R1,548,700.0,W,RUSSELL,E,AVE,39.150670,-77.204063,6D,0,42,10,43,27,10,2019
102,1,Crime Against Society,GERMANTOWN,Other Block Address,GERMANTOWN,MD,20874.0,MCPD,Parking Lot - Commercial,M,5M1,470,19700.0,W,FREDERICK,E,RD,39.178539,-77.238524,5D,0,44,21,26,1,7,2016
108,1,Crime Against Society,WHEATON,Other Block Address,SILVER SPRING,MD,20902.0,MCPD,Other Place,L,4L1,366,23600.0,W,GEORGIA,E,AVE,39.055842,-77.049960,4D,0,15,2,2,8,1,2020
118,1,Crime Against Society,WHEATON,Other Block Address,SILVER SPRING,MD,20902.0,MCPD,Other Place,L,4L1,536,12300.0,W,GEORGIA,E,AVE,39.058066,-77.050032,4D,0,27,20,26,1,7,2016
124,1,Crime Against Property,WHEATON,Other Block Address,SILVER SPRING,MD,20902.0,MCPD,Other Place,L,4L1,360,1100.0,W,UNIVERSITY,W,BLV,39.035857,-77.026918,4D,0,33,14,42,19,10,2019


In [250]:
# Df looks nice, now lets check the value counts for the column
df['Street Name'].value_counts() # Perfect! 

GEORGIA          9665
FREDERICK        6046
VEIRS MILL       5005
NEW HAMPSHIRE    3935
UNIVERSITY       3888
ROCKVILLE        3644
COLESVILLE       2427
RANDOLPH         1738
RUSSELL          1698
Name: Street Name, dtype: int64

In [251]:
## Checking all the others with 'Other'
place = df['Place'].value_counts()
city = df['City'].value_counts()
block = df['Block Address'].value_counts()
div = '-----------'*5
lis = [place,div,city,div,block]

pprint(lis)

[Other Place                    22275
Street - In vehicle             7349
Parking Lot - Commercial        2907
Residence - Apartment/Condo     1703
Other/Unknown                   1215
Street - Residential             662
Residence - Single Family        643
School/College                   566
Parking Lot - Residential        522
Residence - Driveway             129
Residence -Townhouse/Duplex       75
Name: Place, dtype: int64,
 '-------------------------------------------------------',
 SILVER SPRING    22589
ROCKVILLE         4972
GAITHERSBURG      3916
Other Cities      3413
GERMANTOWN        2880
BETHESDA           276
Name: City, dtype: int64,
 '-------------------------------------------------------',
 Other Block Address             28685
11100 BLK  VEIRS MILL RD         3043
20900 BLK  FREDERICK RD          2002
700 BLK  RUSSELL AVE             1302
8600 BLK  COLESVILLE RD           859
8100 BLK  GEORGIA AVE             856
11200 BLK  NEW HAMPSHIRE AVE      683
11200 BLK  GE

In [262]:
# Cleaned_Crime.csv was a bit too large, model and explainer exceeded the max file size for pushing
# and, dash also does not like that.
# checking shape and head again
s = df.shape
print(f"Shape: {s}")
df.head()

Shape: (38046, 27)


,Victims,Crime_Type,Police District Name,Block Address,City,State,Zip Code,Agency,Place,Sector,Beat,PRA,Address Number,Street Prefix,Street Name,Street Suffix,Street Type,Latitude,Longitude,Police District Number,Second,Minute,Hour,Week,Day,Month,Year
99,1,Other,MONTGOMERY VILLAGE,700 BLK RUSSELL AVE,GAITHERSBURG,MD,20877.0,MCPD,Parking Lot - Commercial,R,6R1,548,700.0,W,RUSSELL,E,AVE,39.150670,-77.204063,6D,0,42,10,43,27,10,2019
102,1,Crime Against Society,GERMANTOWN,Other Block Address,GERMANTOWN,MD,20874.0,MCPD,Parking Lot - Commercial,M,5M1,470,19700.0,W,FREDERICK,E,RD,39.178539,-77.238524,5D,0,44,21,26,1,7,2016
108,1,Crime Against Society,WHEATON,Other Block Address,SILVER SPRING,MD,20902.0,MCPD,Other Place,L,4L1,366,23600.0,W,GEORGIA,E,AVE,39.055842,-77.049960,4D,0,15,2,2,8,1,2020
118,1,Crime Against Society,WHEATON,Other Block Address,SILVER SPRING,MD,20902.0,MCPD,Other Place,L,4L1,536,12300.0,W,GEORGIA,E,AVE,39.058066,-77.050032,4D,0,27,20,26,1,7,2016
124,1,Crime Against Property,WHEATON,Other Block Address,SILVER SPRING,MD,20902.0,MCPD,Other Place,L,4L1,360,1100.0,W,UNIVERSITY,W,BLV,39.035857,-77.026918,4D,0,33,14,42,19,10,2019


In [263]:
# Function that applies the logic above for any column which has 'Other' in its name. 
def remove_other(x):
    colP = ['Street - In vehicle', 'Parking Lot - Commercial', 'Residence - Apartment/Condo', 'Street - Residential',
            'Residence - Single Family', 'School/College', 'Parking Lot - Residential', 'Residence - Driveway', 
            'Residence -Townhouse/Duplex']
    
    colC = ['SILVER SPRING', 'ROCKVILLE', 'GAITHERSBURG', 'GERMANTOWN', 'BETHESDA']
    
    colB = ['11100 BLK  VEIRS MILL RD', '20900 BLK  FREDERICK RD', '700 BLK  RUSSELL AVE', '8600 BLK  COLESVILLE RD',
            '8100 BLK  GEORGIA AVE', '8100 BLK  GEORGIA AVE', '11200 BLK  NEW HAMPSHIRE AVE', '11200 BLK  GEORGIA AVE']
    
    colALL = [colP, colC, colB]
    
    
    x = x[x['Place'].isin(colP)]
    x = x[x['City'].isin(colC)]
    x = x[x['Block Address'].isin(colB)]
    
    return x

df = remove_other(df) ### This function reduced my observations to less than 1k from 200k.. 

In [264]:
# Ran the function above to reduce the size of the data
# Now with a shape of (943, 27), that will have to do.
s = df.shape
print(f"Shape: {s}")
df.head()

Shape: (943, 27)


,Victims,Crime_Type,Police District Name,Block Address,City,State,Zip Code,Agency,Place,Sector,Beat,PRA,Address Number,Street Prefix,Street Name,Street Suffix,Street Type,Latitude,Longitude,Police District Number,Second,Minute,Hour,Week,Day,Month,Year
99,1,Other,MONTGOMERY VILLAGE,700 BLK RUSSELL AVE,GAITHERSBURG,MD,20877.0,MCPD,Parking Lot - Commercial,R,6R1,548,700.0,W,RUSSELL,E,AVE,39.150670,-77.204063,6D,0,42,10,43,27,10,2019
610,1,Crime Against Property,WHEATON,11200 BLK GEORGIA AVE,SILVER SPRING,MD,20902.0,MCPD,Parking Lot - Commercial,L,4L2,324,11200.0,S,GEORGIA,E,AVE,39.038668,-77.050882,4D,0,13,14,30,28,7,2019
847,1,Crime Against Person,WHEATON,11100 BLK VEIRS MILL RD,SILVER SPRING,MD,20902.0,MCPD,Parking Lot - Commercial,L,4L2,321,11100.0,S,VEIRS MILL,W,RD,39.037384,-77.051682,4D,0,35,18,8,18,2,2020
848,1,Crime Against Society,WHEATON,11100 BLK VEIRS MILL RD,SILVER SPRING,MD,20902.0,MCPD,Street - In vehicle,L,4L2,321,11100.0,S,VEIRS MILL,W,RD,39.037384,-77.051682,4D,0,35,18,8,18,2,2020
849,1,Crime Against Society,WHEATON,11100 BLK VEIRS MILL RD,SILVER SPRING,MD,20902.0,MCPD,Parking Lot - Commercial,L,4L2,321,11100.0,S,VEIRS MILL,W,RD,39.037384,-77.051682,4D,0,35,18,8,18,2,2020


In [229]:
'''
This was the shape of the data after applying the function above ^^^ (943, 27) <- With function remove_other.
I re-ran the cells and am now at ~38k observations without this last function. 
'''

(943, 27)

## To csv

In [261]:
# Convert cleaned df to csv
df.to_csv('Cleaned_Crime.csv')

In [265]:
# Convert smaller size dataframe to csv
df.to_csv('sml_crime.csv')